# Serving Hyperonym Barba

This Colab notebook contains instructions on how to use a servable Hyperonym Barba model in TensorFlow.

**For online serving, it is recommended to use prebuilt Docker images based on TensorFlow Serving.**

## Mount Google Drive

Mount Google Drive to the local file system:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change working directory into Google Drive:

In [2]:
%mkdir -p /content/drive/MyDrive/hyperonym/barba
%cd /content/drive/MyDrive/hyperonym/barba

/content/drive/MyDrive/hyperonym/barba


## Install dependencies

Install TensorFlow and related libraries:

In [ ]:
!pip install tensorflow==2.11.0 tensorflow-text==2.11.0

In [4]:
import tensorflow as tf
import tensorflow_text as tf_text

## Load servable model

In [5]:
servable = tf.saved_model.load('servables/barba/1')

Flush and unmount Google Drive:

In [6]:
drive.flush_and_unmount()

## Test servable model

In [7]:
def test(hypothesis, premise, signature='serving_default'):
  hypothesis = tf.constant(hypothesis, dtype=tf.string)
  premise = tf.constant(premise, dtype=tf.string)
  signature = servable.signatures[signature]
  return signature(hypothesis=hypothesis, premise=premise)

### Test default signature

In [8]:
test('Leonardo da Vinci never painted on walnut wood', 'The Lady with an Ermine is a portrait painting widely attributed to the Italian Renaissance artist Leonardo da Vinci. Dated to around 1489 to 1491, the work is painted in oils on a panel of walnut wood.')

{'entailment': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00067914], dtype=float32)>}

In [9]:
test('最长的黄鳝能有1米长', '黄鳝（学名：Monopterus albus）：又名鳝鱼，古代称为蝉鱼、黄蝉等。体细长呈蛇形，体长约20-70厘米，最长可达1米。体前圆后部侧扁，尾尖细。')

{'entailment': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.9685478], dtype=float32)>}

### Test cartesian signature

In [10]:
test(['foreign policy', 'Europe', 'elections', 'business', 'outdoor recreation', 'politics'], ['Who are you voting for in 2020?'], signature='cartesian')

{'entailment': <tf.Tensor: shape=(1, 6), dtype=float32, numpy=
 array([[0.16640724, 0.13310751, 0.62789863, 0.09059351, 0.43557516,
         0.21945941]], dtype=float32)>}

In [11]:
test(['融资', '高层变动', '财报公布', '招投标'], ['招商银行完成一轮管理层调整：两位行长助理钟德胜和王小青升任副行长，来自股东中国远洋海运集团有限公司的孙云飞和黄坚将分别出任招行副董事长和非执行董事。'], signature='cartesian')

{'entailment': <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.0985395 , 0.9502329 , 0.18003726, 0.14736123]], dtype=float32)>}